In [1]:
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import paho.mqtt.client as mqtt
from datetime import datetime

In [2]:
# Define connection parameters for InfluxDB
DB_URL = "http://localhost:8086"
DB_TOKEN = "6k_7XcTI53AneCfO1mBZSO3wz7FTZjexthe87XpjRht6ff32RktlIUiZOyXzqN-7S1RZKkyJJroE9n6Zj4C0Lw=="
DB_ORG = "Building"
influx_client = InfluxDBClient(url=DB_URL, token=DB_TOKEN, org=DB_ORG)
writer_api = influx_client.write_api(write_options=SYNCHRONOUS)

In [3]:
# MQTT server setup
MQTT_SERVER = "localhost"
MQTT_PORT = 1883
TOPIC_NAME = "IDF"
SUBSCRIBER_ID = "Building_IDF_Subscriber"
mqtt_client = mqtt.Client(client_id=SUBSCRIBER_ID)

In [4]:
def on_client_connect(mqtt_client, user_data, flags, connect_status):
    print(f"Connected with status code: {connect_status}")
    mqtt_client.subscribe(TOPIC_NAME)

def on_client_message(mqtt_client, user_data, message):
    try:
        datetime_str, zone_temp = message.payload.decode().split(',')
        parsed_datetime = datetime.fromisoformat(datetime_str)
        print(f"Received at: {parsed_datetime}, Dewpoint Temp: {zone_temp}°C")
        temperature_point = Point("Environment")
        temperature_point.field("Average Radiant Temp [C]", float(zone_temp))
        temperature_point.time(parsed_datetime, WritePrecision.NS)
        try:
            writer_api.write(bucket="ICT4BUILDING", org="Building", record=temperature_point)
        except Exception as e:
            print(f"InfluxDB write error: {str(e)}")
    except Exception as e:
        print(f"Message handling error: {str(e)}")

mqtt_client.on_connect = on_client_connect
mqtt_client.on_message = on_client_message

mqtt_client.connect(MQTT_SERVER, MQTT_PORT)
mqtt_client.loop_forever()

Connected with status code: 0
Received at: 2005-01-01 01:00:00+00:00, Dewpoint Temp: 19.221189082648475°C
Received at: 2005-01-01 02:00:00+00:00, Dewpoint Temp: 19.19211336483757°C
Received at: 2005-01-01 03:00:00+00:00, Dewpoint Temp: 19.15997452458384°C
Received at: 2005-01-01 04:00:00+00:00, Dewpoint Temp: 19.12731842111133°C
Received at: 2005-01-01 05:00:00+00:00, Dewpoint Temp: 19.096873529621963°C
Received at: 2005-01-01 06:00:00+00:00, Dewpoint Temp: 19.068515070151165°C
Received at: 2005-01-01 07:00:00+00:00, Dewpoint Temp: 19.120508444372952°C
Received at: 2005-01-01 08:00:00+00:00, Dewpoint Temp: 19.172331513669047°C
Received at: 2005-01-01 09:00:00+00:00, Dewpoint Temp: 19.139495571476328°C
Received at: 2005-01-01 10:00:00+00:00, Dewpoint Temp: 19.102564070906723°C
Received at: 2005-01-01 11:00:00+00:00, Dewpoint Temp: 19.12020897649975°C
Received at: 2005-01-01 12:00:00+00:00, Dewpoint Temp: 19.14500539080575°C
Received at: 2005-01-01 13:00:00+00:00, Dewpoint Temp: 19.25034

KeyboardInterrupt: 